In [1]:
import ast
import folium
import pandas as pd
import numpy as np

In [2]:
#loading the data
data =pd.read_csv('C:\\Users\\mnmureithi\\Downloads\\bts for gis practicals.csv')

In [3]:
#Checking data sample
data.sample(5)

,Longitude,Latitude,BTSName,Classification,Technology
1356,40.927300,2.804800,Buraegarsu,Urban,GSM
1211,37.571389,0.336667,Isiolo_Milimani,Rural,GSM
125,37.076101,-1.036250,Thika_Murathe_Plaza,Urban,GSM
391,37.647583,-0.686556,Kiritiri,Urban,GSM
1322,37.501972,2.005583,Korr_Center,Rural,GSM


In [4]:
#Rows and columns in the dataset
data.shape

(1452, 5)

In [5]:
#Removing duplicates
data.drop_duplicates(subset = ['Longitude' ,'Latitude' ,'Technology'] , keep = 'first', inplace = True)

In [6]:
#Rows and columns in the dataset after removing duplicates
data.shape

(1441, 5)

In [7]:
data.head()

,Longitude,Latitude,BTSName,Classification,Technology
0,40.168942,-1.694245,Masalani,Rural,GSM
1,40.168942,-1.694245,Masalani,Urban,UMTS
3,40.270139,-1.643889,Masalani Quarry,Rural,GSM
4,40.538167,-1.568011,Ijara,Rural,GSM
5,40.038384,-1.497874,Hola_Town,Rural,GSM


In [8]:
data['lat']=data['Latitude']
data['lon']=data['Longitude']

In [9]:
GSM=data[data['Technology']=='GSM']
UMTS=data[data['Technology']=='UMTS']
LTE=data[data['Technology']=='LTE']

In [10]:
LTE.head()

,Longitude,Latitude,BTSName,Classification,Technology,lat,lon
8,40.038384,-1.497874,Hola_Town,Rural,LTE,-1.497874,40.038384
28,37.546389,-1.147351,Matuu_WMX,Rural,LTE,-1.147351,37.546389
39,37.151122,-1.101508,NaN,Rural,LTE,-1.101508,37.151122
63,37.121300,-1.063600,Thika_Flame_Tree,Urban,LTE,-1.063600,37.121300
66,37.149714,-1.062914,Thika_Landless_Estate,Urban,LTE,-1.062914,37.149714


In [11]:
from folium import FeatureGroup, LayerControl, Map, Marker
GSM1 = FeatureGroup(name='GSM')
UMTS1 = FeatureGroup(name='UMTS')
LTE1 = FeatureGroup(name='LTE')

In [12]:
#Obtaining the median to have a reference point
mid_lat=np.median(data['lat'])
mid_long=np.median(data['lon'])

In [13]:
print(mid_lat)
print(mid_long)

-0.42700000000000005
37.46166479


In [14]:
from folium import FeatureGroup, LayerControl, Map, Marker
BTS_map = folium.Map([-0.42700,37.46166479], zoom_start=7, tiles = 'Stamen Terrain', attr = '&copy : @happyMoh')

folium.TileLayer(tiles = 'CartoDB dark_matter', name = 'CartoDB').add_to(BTS_map)
folium.TileLayer(tiles = 'OpenStreetMap', name = 'OpenStreetMap').add_to(BTS_map)
folium.TileLayer(tiles = 'Mapbox Control Room', name = 'Mapbox Ctrl').add_to(BTS_map)
folium.TileLayer(tiles = 'Stamen Watercolor', name = 'Stamen Watercolor').add_to(BTS_map)

for x in range(0,len(GSM)):
    folium.Marker([GSM.iloc[x]['lat'], GSM.iloc[x]['lon']], popup=folium.Popup(GSM.iloc[x]['BTSName']), icon=folium.Icon(color='blue')).add_to(GSM1)
    
for x in range(0,len(UMTS)):
    folium.Marker([UMTS.iloc[x]['lat'], UMTS.iloc[x]['Longitude']], popup=folium.Popup(GSM.iloc[x]['BTSName']), icon=folium.Icon(color='green')).add_to(UMTS1)
    
for x in range(0,len(LTE)):
    folium.Marker([LTE.iloc[x]['lat'], LTE.iloc[x]['Longitude']], popup=folium.Popup(GSM.iloc[x]['BTSName']), icon=folium.Icon(color='purple')).add_to(LTE1)
    
GSM1.add_to(BTS_map)
UMTS1.add_to(BTS_map)
LTE1.add_to(BTS_map)
folium.LayerControl().add_to(BTS_map)

BTS_map.save('D:\\resources\\Digital Academy\\Python\\GIS\\Assignment_3.html')

In [15]:
BTS_map

In [16]:
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from descartes import PolygonPatch
import fiona
from shapely.geometry import Polygon, MultiPolygon, shape
import pandas as pd
import seaborn as sns
bts=data
sns.set()
#Open the shapefile
mp = MultiPolygon(
     [shape(pol['geometry']) for pol in 
fiona.open('D:\\resources\\Digital Academy\\Python\\GIS\\Shapefileke_county.shp') ])
#plot the shape
cm = plt.get_cmap('RdBu')
num_colours = len(mp)
#set the mapsize
fig = plt.figure(figsize=(10, 8), dpi=80)
ax = fig.add_subplot(111)
minx, miny, maxx, maxy = mp.bounds
w, h = maxx - minx, maxy - miny
ax.set_xlim(minx - 0.2 * w, maxx + 0.2 * w)
ax.set_ylim(miny - 0.2 * h, maxy + 0.2 * h)
ax.set_aspect(1)
patches = []
for idx, p in enumerate(mp):
     colour = cm(1. * idx / num_colours)
     patches.append(PolygonPatch(p, fc=colour, ec='#555555', alpha=1., 
zorder=1))
ax.add_collection(PatchCollection(patches, match_original=True))
ax.set_xticks([])
ax.set_yticks([])
sns.scatterplot(x=bts['Longitude'],y=bts['Latitude'],hue=bts['Technology'],data=bts)
plt.title("Kenya Counties and BTS")
plt.savefig('kenya_from_shp.png', alpha=True, dpi=300)
plt.show()

DriverError: D:\resources\Digital Academy\Python\GIS\Shapefileke_county.shp: No such file or directory